In [70]:
import os
import sys
%pylab

import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    %pylab
from src import data

from src import efficient_frontier
from src import filter_assets
from src import plot
from src import portfolio
import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds

Using matplotlib backend: TkAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


c:\Users\Tor Osted\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['plot']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [71]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 0
max_esg_score = 2000
df = pd.read_excel(r"C:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\data\ESG_DATA_S&P500.xlsx")
dates = ['2013-04-01','2014-03-01']
start_year = dates[0]
end_year = dates[1]

Bounds = Bounds(0,1) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.01


In [72]:
esg_data = data.esg_score_weight(df,weights,min_esg_score,max_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data)
prices = prices[['MMM','AOS','ABT']]



1 Failed download:
- ANET: Data doesn't exist for startDate = 1364788800, endDate = 1393650000

1 Failed download:
- CFG: Data doesn't exist for startDate = 1364788800, endDate = 1393650000

1 Failed download:
- CTLT: Data doesn't exist for startDate = 1364788800, endDate = 1393650000

1 Failed download:
- CARR: Data doesn't exist for startDate = 1364788800, endDate = 1393650000

1 Failed download:
- CTVA: Data doesn't exist for startDate = 1364788800, endDate = 1393650000

1 Failed download:
- FOX: Data doesn't exist for startDate = 1364788800, endDate = 1393650000

1 Failed download:
- FTV: Data doesn't exist for startDate = 1364788800, endDate = 1393650000

1 Failed download:
- ETSY: Data doesn't exist for startDate = 1364788800, endDate = 1393650000

1 Failed download:
- HWM: Data doesn't exist for startDate = 1364788800, endDate = 1393650000

1 Failed download:
- HPE: Data doesn't exist for startDate = 1364788800, endDate = 1393650000

1 Failed download:
- GEC: No timezone found,

In [91]:

pd.DataFrame(prices.to_numpy())

,0,1,2
0,104.709999,18.857500,36.919998
1,110.269997,19.600000,36.669998
2,109.349998,18.139999,34.880001
3,117.430000,20.660000,36.630001
4,113.580002,21.025000,33.330002
5,119.410004,22.600000,33.189999
6,125.849998,25.825001,36.549999
7,133.509995,27.075001,38.189999
8,140.250000,26.969999,38.330002
9,128.190002,23.610001,36.660000


In [73]:
esgdata = esgdata[['MMM_weighted','AOS_weighted','ABT_weighted']]

In [74]:
esgdata

,MMM_weighted,AOS_weighted,ABT_weighted
Date,,,
2013-04-01 00:00:00-04:00,1153.0,1158.0,1229.0
2013-05-01 00:00:00-04:00,1153.0,1158.0,1229.0
2013-06-01 00:00:00-04:00,1153.0,1158.0,1229.0
2013-07-01 00:00:00-04:00,1153.0,1158.0,1229.0
2013-08-01 00:00:00-04:00,1153.0,1158.0,1229.0
2013-09-01 00:00:00-04:00,1153.0,1158.0,1229.0
2013-10-01 00:00:00-04:00,1153.0,1158.0,1229.0
2013-11-01 00:00:00-04:00,1153.0,1158.0,1229.0
2013-12-01 00:00:00-05:00,1153.0,1158.0,1229.0


In [92]:

pct_returns = data.pct_returns_from_prices(
pd.DataFrame(prices.to_numpy())) 
pct_returns

,0,1,2
1,0.053099,0.039374,-0.006771
2,-0.008343,-0.074490,-0.048814
3,0.073891,0.138920,0.050172
4,-0.032785,0.017667,-0.090090
5,0.051329,0.074911,-0.004201
6,0.053932,0.142699,0.101235
7,0.060866,0.048403,0.044870
8,0.050483,-0.003878,0.003666
9,-0.085989,-0.124583,-0.043569
10,0.051018,0.052520,0.085106


In [95]:
pct_returns.mean().to_numpy()

array([0.02675008, 0.03115428, 0.0091605 ])

In [77]:
parameters = portfolio.efficient_frontier_solo(pct_returns,Bounds, sharpe_type,start_year,end_year, Wanted_return, maximum_risk)

c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:40: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:113: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


Min. Risk = 5.089% => Return: 2.324%  Sharpe Ratio = 0.46
Max. Sharpe Ratio = 0.52 => Return: 2.68%  Risk: 5.171%


In [78]:
 #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame
weights_of_each_portfolio = efficient_frontier.weights_of_portfolio(prices,parameters)
weights_of_each_portfolio

,MMM,AOS,ABT
0,1.0,0.0,0.0


In [79]:
parameters

[(0.05170623720724132,
  0.02675007612287066,
  0.050890205003251844,
  0.023242085753066157,
  array([0.06115307, 0.06115307, 0.06115307, 0.06115307, 0.06115307,
         0.06115307, 0.06115307, 0.06115307, 0.06115307, 0.06115307,
         0.06115307, 0.06115307, 0.06115307, 0.06115307, 0.06115307,
         0.06115307, 0.06115307, 0.06115307, 0.06115307, 0.06115307,
         0.06115307, 0.06115307, 0.06115307, 0.06115307, 0.06115307,
         0.06115307, 0.06115307, 0.06115307, 0.06115307, 0.06115307,
         0.06115307, 0.06115307, 0.06115307, 0.06115307, 0.06115307,
         0.06115307, 0.06115307, 0.06115307, 0.06115307, 0.06115307,
         0.05951238, 0.05224871, 0.0513877 , 0.08407386, 0.08407386,
         0.08407386, 0.08407386, 0.08407386, 0.08407386, 0.08407386]),
  array([-0.3       , -0.29223979, -0.28447958, -0.27671937, -0.26895917,
         -0.26119896, -0.25343875, -0.24567854, -0.23791833, -0.23015812,
         -0.22239791, -0.2146377 , -0.2068775 , -0.19911729, -0.19

In [80]:
 #Gives us the weights of the individual stocks in our portfolio
portfolio_esg_score = portfolio.esg_score_of_portfolio(weights_of_each_portfolio,esgdata.head(1)) #gives us the ESG score of our portfolio
portfolio_allocations = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[1] #How much we allocate to our market portfolio compared to the risk-free asset
cmle_returns = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[0]

In [83]:
 #Return based on how much we allocate to the risk free asset.
portfolio_esg_score

,ESG_score_of_portfolio
0,1153.0


In [84]:

portfolio_allocations

[1.9340026542483595]

In [81]:
prices[['MMM','AOS','ABT']]

,MMM,AOS,ABT
Date,,,
2013-04-01 00:00:00-04:00,104.709999,18.857500,36.919998
2013-05-01 00:00:00-04:00,110.269997,19.600000,36.669998
2013-06-01 00:00:00-04:00,109.349998,18.139999,34.880001
2013-07-01 00:00:00-04:00,117.430000,20.660000,36.630001
2013-08-01 00:00:00-04:00,113.580002,21.025000,33.330002
2013-09-01 00:00:00-04:00,119.410004,22.600000,33.189999
2013-10-01 00:00:00-04:00,125.849998,25.825001,36.549999
2013-11-01 00:00:00-04:00,133.509995,27.075001,38.189999
2013-12-01 00:00:00-05:00,140.250000,26.969999,38.330002
